# Data

This notebook extracts time-series data from the following parcellation atlases and prepare them for further analysis:

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from src.acnets.pipeline.parcellation import Parcellation

from pathlib import Path

from tqdm.auto import tqdm

In [ ]:
BIDS_DIR = 'data/julia2018'
PARCELLATION_CACHE_DIR = 'data/julia2018/derivatives/resting_timeseries/'

In [ ]:
# data

p = Parcellation(atlas_name='dosenbach2010', bids_dir=BIDS_DIR, cache_dir=PARCELLATION_CACHE_DIR)
dataset = p.fit_transform(None)


In [ ]:
# overall activity distributions across groups (AVGP vs NVGP)

timeseries = dataset['timeseries'].to_dataframe().reset_index()

# TODO normalize across subjects

# h1 regions
h1_regions = timeseries.groupby(['subject', 'region'])['timeseries'].agg(['mean','std']).reset_index()
h1_regions['label'] = h1_regions['subject'].apply(lambda x: x[:4])
h1_regions = h1_regions.melt(id_vars=['subject','label', 'region'], value_vars=['mean','std'])

# for region in tqdm(h1_regions['region'].unique()):
#     features_h1_region = h1_regions.query('region == @region')
#     sns.displot(h1_regions,
#                 x='value', hue='label', col='variable',
#                 kind='kde', fill=True,
#                 facet_kws=dict(sharex=False, sharey=False),
#                 height=3)
#     plt.suptitle(f'H1: {region}')
#     plt.savefig(f'outputs/figures/h1_features/regions/{region}.png')
#     plt.close()

# h1 networks
h1_networks = timeseries.groupby(['subject', 'network'])['timeseries'].agg(['mean','std']).reset_index()
h1_networks['label'] = h1_networks['subject'].apply(lambda x: x[:4])
h1_networks = h1_networks.melt(id_vars=['subject','label', 'network'], value_vars=['mean','std'])

for network in tqdm(h1_networks['network'].unique()):
    features_h1_region = h1_networks.query('network == @network')
    sns.displot(h1_networks,
                x='value', hue='label', col='variable',
                kind='kde', fill=True,
                facet_kws=dict(sharex=False, sharey=False),
                height=3)
    plt.suptitle(f'H1: {network}')
    plt.savefig(f'outputs/figures/h1_features/networks/{network}.png')
    plt.close()
